# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
# File to Load (Remember to Change These)
file_to_load = "Weather.csv"
# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(file_to_load)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
 # Store 'Lat' and 'Lng' into  locations
locations = weather_df[["Lat", "Lng"]].astype(float)
# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
humidity = weather_df["Humidity"].astype(float)

In [31]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
weather_df.drop(weather_df.loc[weather_df['Max Temp']<25].index, inplace=True)
weather_df.reset_index(drop=True)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.60,73.08,29.06,73,62,3.48,MV,1593305202
1,4,saint george,37.10,-113.58,39.00,8,1,4.60,US,1593305035
2,12,hilo,19.73,-155.09,29.00,62,40,4.10,US,1593305000
3,13,torrijos,39.98,-4.28,26.67,49,0,2.10,ES,1593305719
4,15,victoria,22.29,114.16,31.11,70,40,2.68,HK,1593305474
...,...,...,...,...,...,...,...,...,...,...
148,501,araceli,10.55,119.99,26.89,85,100,6.79,PH,1593305760
149,509,marathon,44.90,-89.77,28.00,39,1,2.60,US,1593305761
150,510,nalut,30.33,10.85,29.03,21,0,5.00,LY,1593305761
151,516,vengurla,15.87,73.63,27.26,83,100,0.31,IN,1593305762


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_list = []
weather_df.loc[0]["Lat"]
for i in range(len(weather_df["Lat"])):
    lat = weather_df.iloc[i]['Lat']
    lng = weather_df.iloc[i]['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    try:
        #print(response['results'][0]['name'])
        take2 = (response['results'][0]['name'])
        hotel_list.append(take2)
    except:
        hotel_list.append('')
        #print("Missing value")

In [35]:
hotel_df = weather_df.copy()
hotel_df["Hotel Name"] = hotel_list
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,hithadhoo,-0.60,73.08,29.06,73,62,3.48,MV,1593305202,Scoop Guest House
4,4,saint george,37.10,-113.58,39.00,8,1,4.60,US,1593305035,Best Western Plus Abbey Inn
12,12,hilo,19.73,-155.09,29.00,62,40,4.10,US,1593305000,Hilo Hawaiian Hotel
13,13,torrijos,39.98,-4.28,26.67,49,0,2.10,ES,1593305719,Hotel La Salve
15,15,victoria,22.29,114.16,31.11,70,40,2.68,HK,1593305474,Mini Hotel Central


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel info: {hotel}" for hotel in hotel_info]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))